#Simeon Omeda Project: Optimizing DataFrames and Processing in Chunks

## 1. Introduction 

In this project, we'll practice working with chunked dataframes and optimizing a dataframe's memory usage. We'll be working with financial lending data from Lending Club, a marketplace for personal loans that matches borrowers with investors. You can read more about the marketplace on its website.

The Lending Club's website lists approved loans. Qualified investors can view the borrower's credit score, the purpose of the loan, and other details in the loan applications. Once a lender is ready to back a loan, it selects the amount of money it wants to fund. When the loan amount the borrower requested is fully funded, the borrower receives the money, minus the origination fee that Lending Club charges.

We'll be working with a dataset of loans approved from 2007-2011 (https://bit.ly/3H2XVgC). We've already removed the desc column for you to make our system run more quickly.

If we read in the entire data set, it will consume about 67 megabytes of memory. Let's imagine that we only have 10 megabytes of memory available throughout this project, so you can practice the concepts you learned in the last two lessons.

**Tasks**

1. Read in the first five lines from `loans_2007.csv` (https://bit.ly/3H2XVgC) and look for any data quality issues.

2. Read in the first 1000 rows from the data set, and calculate the total memory usage for these rows. Increase or decrease the number of rows to converge on a memory usage under five megabytes (to stay on the conservative side).

In [1]:
# Read in the first five lines from loans_2007.csv (https://bit.ly/3H2XVgC) and look for any data quality issues
# Importing pandas
import pandas as pd
pd.options.display.max_columns = 99

# Your code goes here
df=pd.read_csv('https://bit.ly/3H2XVgC')
df.head()



/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [2]:
# Read in the first 1000 rows from the data set, and calculate the total memory usage for these rows
df_loans= pd.read_csv("loans_2007.csv", nrows = 1000)
print("usage(MB) for 1000 rows: ",df_loans.memory_usage(deep=True).sum()/(1024**2))

usage(MB) for 1000 rows:  1.5273666381835938


## 2. Exploring the Data in Chunks

Let's familiarize ourselves with the columns to see which ones we can optimize. In the first lesson, we explored column types by reading in the full dataframe. In this project, let's try to understand the column types better while using dataframe chunks.

**Tasks**

For each chunk:
* How many columns have a numeric type? 
* How many have a string type?
* How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?
* Which float columns have no missing values and could be candidates for conversion to the integer type?
* Calculate the total memory usage across all of the chunks.

In [3]:
# Your code goes here
chunk=pd.read_csv('https://bit.ly/3H2XVgC', chunksize=3000)
types=[c.dtypes for c in chunk]



In [4]:
#How many columns have a numeric type?
#How many have a string type?
types[0].value_counts()

float64    30
object     21
int64       1
dtype: int64

In [5]:
# How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?

colObj = types[0].loc[lambda x : x == 'object']
lsObjCols = colObj.index.values.tolist()
lsObjCols

['term',
 'int_rate',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'earliest_cr_line',
 'revol_util',
 'initial_list_status',
 'last_pymnt_d',
 'last_credit_pull_d',
 'application_type']

In [6]:
# How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?

from enum import unique

loan_df = pd.read_csv("loans_2007.csv", chunksize = 2000)

percentage = []
for chunk in loan_df:
  chunk["id"] = pd.to_numeric(chunk["id"], errors = "coerce")
  chunk = chunk.dropna(axis = 0, subset = ["id"])
  unique_string_columns = chunk.select_dtypes(include=["object"]).nunique()
  count_string_columns = chunk.select_dtypes(include=["object"]).count()
  percentage.append((100*unique_string_columns/count_string_columns))
  unique_total = pd.concat(percentage)
  unique_total = unique_total.groupby(unique_total.index).mean()
  category_string_columns = list((unique_total[unique_total<50]).index)

print(print(f"\npercentage of unique values is: \n {unique_total}"))


percentage of unique values is: 
 addr_state              2.410776
application_type        0.056209
earliest_cr_line       18.636656
emp_length              0.632663
emp_title              94.211510
grade                   0.393461
home_ownership          0.193018
initial_list_status     0.056209
int_rate                3.586332
issue_d                 0.337712
last_credit_pull_d      4.069264
last_pymnt_d            3.280709
loan_status             0.194238
purpose                 0.751169
pymnt_plan              0.058481
revol_util             44.077543
sub_grade               1.937759
term                    0.097119
title                  66.508106
verification_status     0.151666
zip_code               25.880729
dtype: float64
None


In [7]:
# Which float columns have no missing values and could be candidates for conversion to the integer type?
colFloat = types[0].loc[lambda x : x == 'float64']
lsFloatCols = colFloat.index.values.tolist()
lsFloatCols

['member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'installment',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'total_acc',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'collections_12_mths_ex_med',
 'policy_code',
 'acc_now_delinq',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'pub_rec_bankruptcies',
 'tax_liens']

In [8]:
# Calculate the total memory usage across all of the chunks.
loan_df = pd.read_csv("loans_2007.csv", chunksize = 2000)
initial_memory = []
for chunk in loan_df:
  initial_memory.append(chunk.memory_usage(deep=True).sum()/(1024**2))

print("total memory usage: {:.4f} MB" . format(sum(initial_memory)))

total memory usage: 65.1909 MB


## 3. Optimizing String Columns

We can achieve the greatest memory improvements by converting the string columns to a numeric type. Let's convert all of the columns where the values are less than 50% unique to the category type, and the columns that contain numeric values to the `float` type.

While working with dataframe chunks:
* Determine which string columns you can convert to a numeric type if you clean them. For example, the `int_rate` column is only a string because of the % sign at the end.
* Determine which columns have a few unique values and convert them to the category type. For example, you may want to convert the grade and `sub_grade` columns.
Based on your conclusions, perform the necessary type changes across all chunks. * Calculate the total memory footprint, and compare it with the previous one.

In [9]:
# Your code goes here
#convert string columns to numeric
convert_columns_dtypes = {"sub_grade":"category", "home_ownership":"category", "verification_status":"category", "purpose":"category"}

loan_df = pd.read_csv("loans_2007.csv", chunksize = 2000, dtype = convert_columns_dtypes, parse_dates = ["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"])
total_memory=[]
for chunk in loan_df:
  term_cleaned=chunk["term"].str.lstrip(" ").str.rstrip("months")
  int_rate_cleaned=chunk["int_rate"].str.rstrip("%")
  revol_cleaned=chunk["revol_util"].str.rstrip("%")
  chunk["term"]=pd.to_numeric(term_cleaned)
  chunk["revol_util"]=pd.to_numeric(revol_cleaned)
  chunk["int_rate"]=pd.to_numeric(int_rate_cleaned)
  total_memory.append(chunk.memory_usage(deep=True).sum()/(1024**2))

print("\nTotal memory usage with string optimisation: {:.2f} MB" . format(sum(total_memory)))



Total memory usage with string optimisation: 38.80 MB


## 4. Optimizing Numeric Columns

It looks like we were able to realize some powerful memory savings by converting to the category type and converting string columns to numeric ones.

Now let's optimize the numeric columns using the `pandas.to_numeric()` function.

**Tasks**

While working with dataframe chunks:
* Identify float columns that contain missing values, and that we can convert to a more space efficient subtype.
* Identify float columns that don't contain any missing values, and that we can convert to the integer type because they represent whole numbers.
* Based on your conclusions, perform the necessary type changes across all chunks.
* Calculate the total memory footprint and compare it with the previous one.




In [ ]:
df = pd.read_csv("loans_2007.csv", chunksize = 2000, dtype = convert_columns_dtypes, parse_dates = ["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"])
import numpy as np
def change_to_int(df, column_name):
  max_column=df[column_name].max()
  min_column=df[column_name].min()
  for dtype_name in ["int8", "int16", "int32", "int64"]:
    if max_column < np.iinfo(dtype_name).max and min_column > np.iinfo(dtype_name).min:
      df[column_name] = df[column_name].astype(dtype_name)
      break

total_memory = []
for chunk in df:
  chunk["id"] = pd.to_numeric(chunk["id"], errors = "coerce")
  chunk = chunk.dropna(axis=0, subset = ["id"])
  term_cleaned=chunk["term"].str.lstrip(" ").str.rstrip(" months")
  int_rate_cleaned=chunk["int_rate"].str.rstrip("%")
  revol_cleaned=chunk["revol_util"].str.rstrip("%")
  chunk["term"]=pd.to_numeric(term_cleaned)
  chunk["revol_util"]=pd.to_numeric(revol_cleaned)
  chunk["int_rate"]=pd.to_numeric(int_rate_cleaned)
  float_columns=chunk.select_dtypes(include=["float"])
  float_columns=float_columns.dropna()
  for columns in float_columns.columns:
    if columns in missing:
      chunk[columns]=pd.to_numeric(chunk[columns], downcast="float")
    elif columns in no_missing:
      change_to_int(chunk, columns)
  total_memory.append(chunk.memory_usage(deep=True).sum()/(1024**2))

print("\nTotal memory usage: {:.2f} MB" . format(sum(total_memory)))
print("\nPercentage memory savings: {:.2f} %" .format(100*(sum(initial_memory) - sum(total_memory))/sum(initial_memory)))
print(f"\n {chunk.dtypes}")

## Next Steps

We've practiced optimizing a dataframe's memory footprint and working with dataframe chunks. Here's an idea for some next steps:

Create a function that automates as much of the work you just did as possible, so that you could use it on other Lending Club data sets. This function should:

* Determine the optimal chunk size based on the memory constraints you provide.

* Determine which string columns can be converted to numeric ones by removing the `%` character.

* Determine which numeric columns can be converted to more space efficient representations.


In [14]:
# Your code goes here
def optimizer(file,mem_limit, chunk_step):

SyntaxError: ignored

In [15]:
# Determine the optimal chunk size based on the memory constraints you provide.

def optimal_chunk_size(csv_file, desired_mem, row_steps, start_chunk_size):
  no_rows = start_chunk_size
  chunk_memory =0
  while(chunk_memory < desired_mem):
    no_rows += row_steps
    chunk = pd.read_csv(csv_file, nrows=no_rows)
    chunk_memory = chunk.memory_usage(deep=True).sum()/(1024**2)
  return(no_rows - row_steps) # reduce by 1

optimal_chunk_size('https://bit.ly/3H2XVgC', 5, 50, 2500)

3250

In [16]:
# Determine which string columns can be converted to numeric ones by removing the % character
def string_to_numeric(df, character):
  columns_with_char = []
  for key, value in df.iteritems():
    if  value.str.contains(character).any(): #check if column has % character
      try:
        value = value.str.replace('%', '')  #remove it and try to convert to numeric
        value = pd.to_numeric(value, errors='raise') 
        columns_with_char.append(key)
      except(ValueError):
        #dont add columns to the list
        continue
  print(columns_with_char)

loans = pd.read_csv('https://bit.ly/3H2XVgC')
string_cols = loans.select_dtypes(include='object')
string_to_numeric(string_cols, '%')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


['int_rate', 'revol_util']


In [17]:
#Determine which numeric columns can be converted to more space efficient representations

def numeric_mem_optimise(df): 
  
  numeric_cols = df.select_dtypes(exclude=['object'])
  cols_missing_value = numeric_cols.isnull().sum()
 
  int_cols = list(cols_missing_value[cols_missing_value == 0].index)
  float_cols = list(cols_missing_value[cols_missing_value >0].index)

  return int_cols, float_cols

loans = pd.read_csv('https://bit.ly/3H2XVgC')
loans['id']=pd.to_numeric(loans['id'], errors='coerce')
loans.dropna(axis=0, subset=['id'], inplace=True)

x,y = numeric_mem_optimise(loans)
print(f'int columns: {x}')
print(f'float columns: {y}')

int columns: ['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'installment', 'dti', 'revol_bal', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'policy_code']
float columns: ['annual_inc', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'total_acc', 'collections_12_mths_ex_med', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'pub_rec_bankruptcies', 'tax_liens']
